In [1]:
#imports
import requests
import json
import pandas as pd
import numpy as np
import warnings
import random
import matplotlib.pyplot as plt
from datetime import datetime
import time as clock


In [2]:

Sensors_Data = pd.read_json('data/W512_readings.json')
Aircon_Data = pd.read_json('data/W512_aircon_status.json')
Weather_Data = pd.read_json('data/Weather_data.json')


# Normalize the data
Aircon_rows = []

for _, row in Aircon_Data.iterrows():
    date = row['date']
    time = row['time']
    
    flattened_row = {
        "date": date,
        "time": time
    }
    
    fc_readings = row['FC_FullStatus_Readings']
    
    for unit, data in fc_readings.items():
        flattened_row[f"{unit}_Status"] = data.get("Status", None)
        flattened_row[f"{unit}_Fan_Status"] = data.get("Fan_Status", None)
        flattened_row[f"{unit}_Set_Point"] = data.get("Set_Point", None)
        flattened_row[f"{unit}_Operation_Mode"] = data.get("Operation_Mode", None)
    
    Aircon_rows.append(flattened_row)

Sensors_rows = []
include_keys_1 = ["24E124725E285123", "24E124725E331695","24E124725E331744",
                      "24E124725E332483","24E124725E290348","24E124725E331733","24E124725E286745"]#"24E124136D316361" is suppiosed to be outdoor but it is not outdoor yet
include_keys_2 = ["Sensor_1","Sensor_3","Sensor_6"]
for _, row in Sensors_Data.iterrows():
    invalid_input = False
    
    date = row['date']
    time = row['time']
    
    flattened_row = {
        "date": date,
        "time": time
    }
    
    
    lorawan_readings = row['Lorawan_Readings']
    
    if isinstance(lorawan_readings, dict):
        for unit, data in lorawan_readings.items():
            if unit not in include_keys_1:
                continue
            if isinstance(data, dict):  # Ensure that each item in Lorawan_Readings is a dictionary
                for key, value in data.items():
                    flattened_row[f"{unit}_{key}"] = value
            
    energy_readings = row['Energy_Readings']
    total_power = 0
    total_energy = 0
    invalid_input_power = False
    invalid_input_energy = False
    
    for unit, data in energy_readings.items():
        if unit not in include_keys_2:
            continue
        power = data.get('Power', None)
        energy = data.get('Energy', None)
        if power is None:
            invalid_input_power = True
        if energy is None:
            invalid_input_energy = True
        total_power += power
        total_energy += energy
        
    if invalid_input_power:
        total_power = None
    if invalid_input_energy:
        total_energy = None
        
    flattened_row["Total_Energy"] = total_energy
    flattened_row["Total_Power"] = total_power
    
    Sensors_rows.append(flattened_row)


# Normalize the data
Weather_rows = []

for _, row in Weather_Data.iterrows():
    date = row['date']
    time = row['time']
    
    flattened_row = {
        "date": date,
        "time": time
    }
    
    flattened_row['weather_status']= row['result']['weather_status']
    flattened_row['weather_temp']= row['result']['weather_temp']
    flattened_row['weather_humidity']= row['result']['weather_humidity']
    
    Weather_rows.append(flattened_row)
    
print("Normalizing data")



Aircon_Normalize_Data = pd.DataFrame(Aircon_rows)
Sensors_Normalize_Data = pd.DataFrame(Sensors_rows)
Weather_Normalize_Data = pd.DataFrame(Weather_rows)
# For Aircon_Normalize_Data
Aircon_Normalize_Data['datetime_str'] = Aircon_Normalize_Data['date'].astype(str) + ' ' + Aircon_Normalize_Data['time']
Aircon_Normalize_Data['datetime'] = Aircon_Normalize_Data['datetime_str'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %I:%M:%S %p"))
Aircon_Normalize_Data['timestamp'] = Aircon_Normalize_Data['datetime'].apply(lambda x: int(x.timestamp()))

# For Sensors_Normalize_Data
Sensors_Normalize_Data['datetime_str'] = Sensors_Normalize_Data['date'].astype(str) + ' ' + Sensors_Normalize_Data['time']
Sensors_Normalize_Data['datetime'] = Sensors_Normalize_Data['datetime_str'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %I:%M:%S %p"))
Sensors_Normalize_Data['timestamp'] = Sensors_Normalize_Data['datetime'].apply(lambda x: int(x.timestamp()))

# For Weather_Normalize_Data
Weather_Normalize_Data['datetime_str'] = Weather_Normalize_Data['date'].astype(str) + ' ' + Weather_Normalize_Data['time']
Weather_Normalize_Data['datetime'] = Weather_Normalize_Data['datetime_str'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %I:%M:%S %p"))
Weather_Normalize_Data['timestamp'] = Weather_Normalize_Data['datetime'].apply(lambda x: int(x.timestamp()))

merged_data = pd.merge_asof(
    Aircon_Normalize_Data,  # Left DataFrame
    Sensors_Normalize_Data,      # Right DataFrame
    on='timestamp',   # Key column
    direction='nearest'    # Match the nearest time
)
merged_data = pd.merge_asof(
    merged_data,  # Left DataFrame
    Weather_Normalize_Data,      # Right DataFrame
    on='timestamp',   # Key column
    direction='nearest'    # Match the nearest time
)

temperature_col = [
    col for col in merged_data.columns 
    if "temperature" in col.lower()
]
humidity_col = [
    col for col in merged_data.columns 
    if "humidity" in col.lower()
]

def get_unit_columns(unit_number, columns):
    return [col for col in columns if f"FC_Unit_{unit_number}" in col]

aircon_units = len([
    col for col in merged_data.columns
    if "FC_Unit_" in col and "_Status" in col and "Fan" not in col
])

aircon_units_cols = {}

for unit in range(1, aircon_units + 1):
    aircon_units_cols[f'Unit_{unit}'] = get_unit_columns(unit, merged_data.columns)


final_data = pd.DataFrame()
final_data["timestamp"] = merged_data["timestamp"]

final_data["temperature"] = merged_data[temperature_col].apply(lambda x: round(x.mean(), 3), axis=1)
final_data["humidity"] = merged_data[humidity_col].apply(lambda x: round(x.mean(),3), axis=1)

final_data['power_consumption'] = merged_data['Total_Power']
final_data['energy_consumption'] = merged_data['Total_Energy']

for unit, columns in aircon_units_cols.items():
    for column in columns:
        if 'set_point' in column:
            final_data[column] = merged_data[column].replace(0, pd.NA).ffill()
        else:
            final_data[column] = merged_data[column].replace("ERROR", pd.NA).ffill()

final_data.dropna(inplace=True)
# final_data.reset_index(drop=True, inplace=True)#######################################


final_data.to_csv('final_data_W512_A_star.csv', index=False)

def getFCData(data, row_index):
    settings = []
    for i in range(1, aircon_units + 1):
        settings.append(data[f"FC_Unit_{i}_Status"].iloc[row_index])
        settings.append(data[f"FC_Unit_{i}_Fan_Status"].iloc[row_index])
        settings.append(data[f"FC_Unit_{i}_Set_Point"].iloc[row_index])
        settings.append(data[f"FC_Unit_{i}_Operation_Mode"].iloc[row_index])
        
    return settings

def is_same_settings(data, curr_row_index, next_row_index):   
    return True if (getFCData(data, curr_row_index) == getFCData(data, next_row_index)) else False


def is_all_off(data, curr_row_index, check_for_off):
    for i in range(1, aircon_units + 1):
        if data[f"FC_Unit_{i}_Status"].iloc[curr_row_index] == "ON":
            return not check_for_off
        
    return check_for_off



aircon_status_result = pd.DataFrame()
total_final_rows = final_data.shape[0]
Aircon_Normalize_Data = Aircon_Normalize_Data.drop(['date', 'time', 'datetime_str', 'datetime', 'timestamp'], axis=1)
for i in range(total_final_rows - 1, -1, -1):

    if is_all_off(final_data, i, True):
        continue

    rows = []
    time_taken = []
    energy_consumption = []
    previous_temp = []
    previous_humi = []
    
    curr_timestamp = final_data["timestamp"].iloc[i]
    curr_energy = final_data["energy_consumption"].iloc[i]
    curr_temperature = final_data["temperature"].iloc[i]
    curr_humidity = final_data["humidity"].iloc[i]

    
    while i >= 0 and is_same_settings(final_data, i - 1, i):
        timetaken = curr_timestamp - final_data["timestamp"].iloc[i - 1]
        if timetaken < 15 or timetaken > 3600:
            break
        rows.append(i - 1)
        time_taken.append(timetaken)
        energy_consumption.append(curr_energy - final_data["energy_consumption"].iloc[i - 1])
        previous_temp.append(final_data["temperature"].iloc[i - 1])
        previous_humi.append(final_data["humidity"].iloc[i - 1])
        
        i -= 1
        
    temp_df = pd.DataFrame({
            'timestamp': [curr_timestamp],
            'rows': [rows],
            'time_taken': [time_taken],
            'energy_consumption': [energy_consumption],
            'previous_temp': [previous_temp],
            'previous_humi': [previous_humi],
            'current_temp': [curr_temperature],
            'current_humi': [curr_humidity],
        })
    for col in Aircon_Normalize_Data.columns:
        temp_df[col] = final_data[col].iloc[i]
    
        
    aircon_status_result = pd.concat([aircon_status_result, temp_df], ignore_index=False)        
        
        
        
        
print("Finished")
aircon_status_result = aircon_status_result.sort_values(by=['current_temp'], ascending=False)
aircon_status_result.to_csv('aircon_status_W512_A_star.csv', index=False)
aircon_status_result.info()




Normalizing data
Finished
<class 'pandas.core.frame.DataFrame'>
Index: 5517 entries, 0 to 0
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   timestamp                 5517 non-null   int64  
 1   rows                      5517 non-null   object 
 2   time_taken                5517 non-null   object 
 3   energy_consumption        5517 non-null   object 
 4   previous_temp             5517 non-null   object 
 5   previous_humi             5517 non-null   object 
 6   current_temp              5517 non-null   float64
 7   current_humi              5517 non-null   float64
 8   FC_Unit_1_Status          5517 non-null   object 
 9   FC_Unit_1_Fan_Status      5517 non-null   object 
 10  FC_Unit_1_Set_Point       5517 non-null   float64
 11  FC_Unit_1_Operation_Mode  5517 non-null   object 
 12  FC_Unit_2_Status          5517 non-null   object 
 13  FC_Unit_2_Fan_Status      5517 non-null   obj

2367
2366
2365
2364
2363
2362
2361
2360
2359
2358
2357
2356
2355
2354
2353
2352
2351
2350
2349
2348
2347
2346
2345
2344
2343
2342
2341
2340
2339
2338
2337
2336
2335
2334
2333
2332
2331
2330
2329
2328
2327
2326
2325
2324
2323
2322
2321
2320
2319
2318
2317
2316
2315
2314
2313
2312
2311
2310
2309
2308
2307
2306
2305
2304
2303
2302
2301
2300
2299
2298
2297
2296
2295
2294
2293
2292
2291
2290
2289
2288
2287
2286
2285
2284
2283
2282
2281
2280
2279
2278
2277
2276
2275
2274
2273
2272
2271
2270
2269
2268
2267
2266
2265
2264
2263
2262
2261
2260
2259
2258
2257
2256
2255
2254
2253
2252
2251
2250
2249
2248
2247
2246
2245
2244
2243
2242
2241
2240
2239
2238
2237
2236
2235
2234
2233
2232
2231
2230
2229
2228
2227
2226
2225
2224
2223
2222
2221
2220
2219
2218
2217
2216
2215
2214
2213
2212
2211
2210
2209
2208
2207
2206
2205
2204
2203
2202
2201
2200
2199
2198
2197
2196
2195
2194
2193
2192
2191
2190
2189
2188
2187
2186
2185
2184
2183
2182
2181
2180
2179
2178
2177
2176
2175
2174
2173
2172
2171
2170
2169
2168


652
651
650
649
648
647
646
645
644
643
642
641
640
639
638
637
636
635
634
633
632
631
630
629
628
627
626
625
624
623
622
621
620
619
618
617
616
615
614
613
612
611
610
609
608
607
606
605
604
603
602
601
600
599
598
597
596
595
594
593
592
591
590
589
588
587
586
585
584
583
582
581
580
579
578
577
576
575
574
573
572
571
570
569
568
567
566
565
564
563
562
561
560
559
558
557
556
555
554
553
552
551
550
549
548
547
546
545
544
543
542
541
540
539
538
537
536
535
534
533
532
531
530
529
528
527
526
525
524
523
522
521
520
519
518
517
516
515
514
513
512
511
510
509
508
507
506
505
504
503
502
501
500
499
498
497
496
495
494
493
492
491
490
489
488
487
486
485
484
483
482
481
480
479
478
477
476
475
474
473
472
471
470
469
468
467
466
465
464
463
462
461
460
459
458
457
456
455
454
453
452
451
450
449
448
447
446
445
444
443
442
441
440
439
438
437
436
435
434
433
432
431
430
429
428
427
426
425
424
423
422
421
420
419
418
417
416
415
414
413
412
411
410
409
408
407
406
405
404
403


In [3]:
target_temp_range = np.arange(20, 29.5, 0.5)
time_factor = 0.5
energy_factor = 0.5
acceptable_range = 0.8
total_rows = aircon_status_result.shape[0]
stored_dictionary = {}


def getRowData(row_index):
    temperature = aircon_status_result["current_temp"].iloc[row_index]
    humidity = aircon_status_result["current_humi"].iloc[row_index]
    
    return [temperature, humidity]

def getArrayData(row_index, array_index):
    time_taken = aircon_status_result["time_taken"].iloc[row_index]
    energy_consumption = aircon_status_result["energy_consumption"].iloc[row_index]
    temperature = aircon_status_result["previous_temp"].iloc[row_index]
    humidity = aircon_status_result["previous_humi"].iloc[row_index]
    
    return [temperature[array_index], humidity[array_index], time_taken[array_index], energy_consumption[array_index]]

def comparePath(best_path, current_path):    
    if best_path['factor'] > current_path['factor']:
        return True
    
    return False

# Get heuristic score
h_score = 0
total_score = 0
total_checked = 0
for i in range(total_rows):
    row_data = getRowData(i)
    for j in range(len(aircon_status_result['rows'].iloc[i])):
        array_data = getArrayData(i, j)
        if (abs(row_data[0] - array_data[0]) > 0):
            total_score += ((array_data[3] * energy_factor) + (array_data[2] * time_factor))/(abs(row_data[0] - array_data[0]))
            total_checked += 1
        
h_score = total_score/total_checked
print(h_score)

# h_score is the average amount of energy and time is needed for 1 degree celius

32470.12099193978


In [ ]:
start_time = clock.time()


def calculateScore(array_data):
    return (array_data[3] * energy_factor) + (array_data[2] * time_factor)

def calculateHeuristicScore(start_temp, end_temp):
    return abs(end_temp - start_temp) * h_score

def getSettingWithLowestScore(target_temp, current_row, lowest_score, checked_nodes):
    # return the row with the best score
    out_of_range = True
    setting_with_lowest = [-1, -1]
    for i in range(current_row + 1, total_rows):
        row_data = getRowData(i)
        if abs(row_data[0] - target_temp) > acceptable_range:
            if not out_of_range:
                break
            continue
        
        out_of_range = False
        # Current row temperature is in range
        for j in range(len(aircon_status_result['rows'].iloc[i])):
            if checked_nodes[i][j]:
                continue
            array_data = getArrayData(i, j)
            heuristicScore = calculateScore(array_data) + calculateHeuristicScore(array_data[0], target_temp)

            if heuristicScore < lowest_score:
                setting_with_lowest = [i, j]
                lowest_score = heuristicScore
    
    return setting_with_lowest

for target_temp in target_temp_range:
    paths = {}
    
    def AStar(curr_row_index, curr_array_index):
        rowData = getRowData(curr_row_index)
        arrayData = getArrayData(curr_row_index, curr_array_index)
        
        if abs(arrayData[0] - target_temp) < acceptable_range:
            path = {
                'energy_consumption': [arrayData[3]],
                'time_taken': [arrayData[2]],
                'factor': array_data[3] * energy_factor + array_data[2] * time_factor,
                'starting_temp': rowData[0],
                'starting_humi': rowData[1],
                'ending_temp': arrayData[0],
                'ending_humi': arrayData[1],
                'path': [curr_row_index]
            }
            
            paths[curr_row_index] = path
            hueristicScore = calculateScore(arrayData)
            
            return hueristicScore, path
        
        path = []
        
        while True:
            lowestHeuristicScore = float("inf")
            lowestScoreIndex = -1
            lowestArrayIndex = -1
            
            checked_nodes = [[False] * len(aircon_status_result['rows'].iloc[i]) for i in range(total_rows)]
            indexes = getSettingWithLowestScore(target_temp, curr_row_index, lowestHeuristicScore, checked_nodes)
            
            if indexes[0] != -1:
                lowestScoreIndex = indexes[0]
                lowestArrayIndex = indexes[1]
            elif lowestScoreIndex == -1:
                # No available paths
                break
            
            
            if checked_nodes[lowestScoreIndex][lowestArrayIndex]:
                # Best path
                curr_path = {
                    'energy_consumption': [array_data[3]] + path['energy_consumption'],
                    'time_taken': [array_data[2]] + path['time_taken'],
                    'starting_temp': rowData[0],
                    'starting_humi': rowData[1],
                    'ending_temp': path['ending_temp'],
                    'ending_humi': path['ending_humi'],
                    'path': [curr_row_index] + path['path']
                }
                
                curr_path['factor'] = sum(curr_path['energy_consumption']) * energy_factor + sum(curr_path['time_taken']) * time_factor
                paths[curr_row_index] = curr_path
                return lowestHeuristicScore, curr_path
                
            lowestArrayData = getArrayData(lowestScoreIndex, lowestArrayIndex)
            
            checked_nodes[lowestScoreIndex][lowestArrayIndex] = True
            tempScore, path = AStar(lowestScoreIndex, lowestArrayIndex)
            lowestHeuristicScore = calculate(lowestArrayData) + tempScore 
            
            
        # no valid paths
        paths[curr_row_index] = {
            'energy_consumption': [], 
            'starting_temp': row_data[0], 
            'starting_humi': row_data[1],
            'ending_temp': 0,
            'ending_humi': 0,
            'time_taken': [], 
            'factor': float('inf'), 
            'path': []
        }
        
        return float("inf"), paths[curr_row_index]
                                     
    for i in range(total_rows):
        if i not in paths:
            print(i)
            lowestHeuristicScore = float("inf")
            lowestScoreIndex = -1
            rowData = getRowData(i)
            path = []
            
            checked_nodes = [False] * len(aircon_status_result['rows'].iloc[i])
            # Check through all the nodes and find the smallest heuristic value
            while True:
                for j in range(len(aircon_status_result['rows'].iloc[i])):
                    if checked_nodes[j]:
                        continue
                    
                    arrayData = getArrayData(i, j)
                    # Check nodes for if they have a lower heuristical score
                    heuristicScore = calculateScore(arrayData) + calculateHeuristicScore(rowData[0], target_temp)
                    if heuristicScore < lowestHeuristicScore:
                        lowestHeuristicScore = heuristicScore
                        lowestScoreIndex = j
                        
                if lowestScoreIndex == -1:
                    break
                
                # if checked is still False, the current is the optimized
                array_data = getArrayData(i, lowestScoreIndex)
                if checked_nodes[lowestScoreIndex]:
                    # If valid path
                    if path and path["energy_consumption"]:
                        paths[i] = {
                            'energy_consumption': [array_data[3]] + path['energy_consumption'],
                            'time_taken': [array_data[2]] + path['time_taken'],
                            'starting_temp': array_data[0],
                            'starting_humi': array_data[1],
                            'ending_temp': path['ending_temp'],
                            'ending_humi': path['ending_humi'],
                            'path': [i] + path['path'] 
                        }
                        paths[i]['factor'] = sum(paths[i]['energy_consumption']) * energy_factor + sum(paths[i]['time_taken']) * time_factor
                    else:
                        # Invalid path
                        paths[i] = {
                            'energy_consumption': [], 
                            'starting_temp': row_data[0], 
                            'starting_humi': row_data[1],
                            'ending_temp': 0,
                            'ending_humi': 0,
                            'time_taken': [], 
                            'factor': float('inf'), 
                            'path': []
                        }
                        
                        
                    # Found the most optimal path, exit loop
                    break
                    
                checked_nodes[lowestScoreIndex] = True
                tempScore, path = AStar(i, lowestScoreIndex)
                lowestHeuristicScore = calculateScore(array_data) + tempScore 
                
    print(paths)
    # Select the best alternative if no valid paths exist
    if not any(path['factor'] < float('inf') for path in paths.values()):
        print(f"Target temp {target_temp} not achievable, selecting next best value.")
        best_alternative = min(paths.values(), key=lambda p: p['factor'])
        stored_dictionary[f"Dictionary for target temp: {target_temp}"] = best_alternative
    else:
        stored_dictionary[f"Dictionary for target temp: {target_temp}"] = paths.copy()
        
end_time = clock.time()
elapsed_time = end_time - start_time
elapsed_time_minutes = elapsed_time / 60
print(f"Processing time: {elapsed_time_minutes:.2f} minutes")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
#asssuming there's multiple key for target temp
unique_keys = [key for key in stored_dictionary.keys() if key.startswith("Dictionary for target temp") or key.startswith("Best alternative for target temp")]
print(unique_keys)

current_temperature = 28
target_temp = 25
key_to_find = f"Dictionary for target temp: {target_temp}"




def expandPath(row_index):
    for i in range(1, aircon_units + 1):
        print("UNIT " + str(i) + ":", end=' ')
        print(aircon_status_result[f"FC_Unit_{i}_Status"].iloc[row_index], end=' ')
        print(aircon_status_result[f"FC_Unit_{i}_Fan_Status"].iloc[row_index], end=' ')
        print(aircon_status_result[f"FC_Unit_{i}_Set_Point"].iloc[row_index], end=' ')
        print(aircon_status_result[f"FC_Unit_{i}_Operation_Mode"].iloc[row_index], end=' ')
        print("")
        
def findClosestTemperature(current_temp, paths):
    """Find the index of the closest available temperature in the paths dictionary."""
    closest_temp_index = None
    smallest_difference = float('inf')
    
    for key, value in paths.items():
        if value['starting_temp'] and value['path']:  #ensure that paths[index] has both values in 'starting_temp' and 'path'
            difference = abs(value['starting_temp'] - current_temp)
            if difference < smallest_difference:
                smallest_difference = difference
                closest_temp_index = key   
    return closest_temp_index


# Check if either key exists in stored_dictionary
if key_to_find in stored_dictionary:
    stored_dict_key = stored_dictionary[key_to_find]


    filtered_paths = {
        key: value for key, value in stored_dict_key.items()
        if (abs(value['starting_temp'] - current_temperature) < acceptable_range)
    }
    
    if filtered_paths:
        # Find the path with the smallest factor
        smallest_factor_path = min(filtered_paths.keys(), key=lambda x: filtered_paths[x]['factor'])
        print(stored_dict_key[smallest_factor_path])
    
        for index, value in enumerate(stored_dict_key[smallest_factor_path]['path']):
            expandPath(value)
            print("For", end=" ")
            hours, remainder = divmod(stored_dict_key[smallest_factor_path]['time_taken'][index], 3600)
            minutes, seconds = divmod(remainder, 60)
            print(f"Hours: {hours}, Minutes: {minutes}, Seconds: {seconds}")
            print("")
    else:
        print("No paths found within the acceptable range.")
        print("Finding the closest temperature in the algorithm.")
    
        closest_temp_index = findClosestTemperature(current_temperature, paths)
        
        if closest_temp_index is not None:
            
            print("Closest temperature found at index:", closest_temp_index , "\n")
            print(paths[closest_temp_index])
            
            for index, value in enumerate(stored_dict_key[closest_temp_index]['path']):
                expandPath(value)
                print("For", end=" ")
                hours, remainder = divmod(stored_dict_key[closest_temp_index]['time_taken'][index], 3600)
                minutes, seconds = divmod(remainder, 60)
                print(f"Hours: {hours}, Minutes: {minutes}, Seconds: {seconds}")
                print("")
        else:
            print("No valid paths available, even for the closest temperature.")

else:
    print(f"{key_to_find} does not exist")





In [ ]:
# unique_keys = [key for key in stored_dictionary.keys() if key.startswith("Dictionary for target temp") or key.startswith("Best alternative for target temp")]
# print(unique_keys)

unique_keys = set()

